# Plot

> Creating estimation plots.

- order: 4

In [ ]:
#| default_exp plotter

In [ ]:
#| hide
from __future__ import annotations

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev
nbdev.nbdev_export()

In [ ]:
#| export
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import warnings
import logging

In [ ]:
# | export
# TODO refactor function name
def effectsize_df_plotter(effectsize_df, **plot_kwargs):
    """
    Custom function that creates an estimation plot from an EffectSizeDataFrame.
    Keywords
    --------
    Parameters
    ----------
    effectsize_df
        A `dabest` EffectSizeDataFrame object.
    plot_kwargs
        color_col=None
        raw_marker_size=6, es_marker_size=9,
        swarm_label=None, contrast_label=None, delta2_label=None,
        swarm_ylim=None, contrast_ylim=None, delta2_ylim=None,
        custom_palette=None, swarm_desat=0.5, halfviolin_desat=1,
        halfviolin_alpha=0.8,
        face_color = None,
        bar_label=None, bar_desat=0.8, bar_width = 0.5,bar_ylim = None,
        ci=None, ci_type='bca', err_color=None,
        float_contrast=True,
        show_pairs=True,
        show_delta2=True,
        group_summaries=None,
        group_summaries_offset=0.1,
        fig_size=None,
        dpi=100,
        ax=None,
        gridkey_rows=None,
        swarmplot_kwargs=None,
        violinplot_kwargs=None,
        slopegraph_kwargs=None,
        sankey_kwargs=None,
        reflines_kwargs=None,
        group_summary_kwargs=None,
        legend_kwargs=None,
        title=None, fontsize_title=16,
        fontsize_rawxlabel=12, fontsize_rawylabel=12,
        fontsize_contrastxlabel=12, fontsize_contrastylabel=12,
        fontsize_delta2label=12,
        swarm_bars=True, swarm_bars_kwargs=None,
        contrast_bars=True, contrast_bars_kwargs=None,
        delta_text=True, delta_text_kwargs=None,
        delta_dot=True, delta_dot_kwargs=None,
    """
    from .misc_tools import (get_params,
                             get_kwargs,
                             get_color_palette,
                             initialize_fig,
                             get_plot_groups,
                             add_counts_to_ticks,
                             extract_contrast_plotting_ticks,
    )
    from .plot_tools import (
        get_swarm_spans,
        error_bar,
        sankeydiag,
        swarmplot,
        swarm_bars_plotter,
        contrast_bars_plotter,
        summary_bars_plotter,
        delta_text_plotter,
        DeltaDotsPlotter,
        slopegraph_plotter,
        plot_minimeta_or_deltadelta_violins,
        effect_size_curve_plotter,
        grid_key_WIP,
        barplotter,
    )
    from ._stats_tools.effsize import (
        _compute_standardizers,
        _compute_hedges_correction_factor,
    )

    warnings.filterwarnings(
        "ignore", "This figure includes Axes that are not compatible with tight_layout"
    )

    # Have to disable logging of warning when get_legend_handles_labels()
    # tries to get from slopegraph.
    logging.disable(logging.WARNING)

    # Save rcParams that I will alter, so I can reset back.
    original_rcParams = {}
    _changed_rcParams = ["axes.grid"]
    for parameter in _changed_rcParams:
        original_rcParams[parameter] = plt.rcParams[parameter]

    plt.rcParams["axes.grid"] = False
    ytick_color = plt.rcParams["ytick.color"]

    # Extract parameters and set kwargs
    (face_color, dabest_obj, plot_data, xvar, yvar, is_paired, effect_size, 
     proportional, all_plot_groups, idx, show_delta2, show_mini_meta, 
     float_contrast, show_pairs, effect_size_type, group_summaries, err_color) = get_params(effectsize_df=effectsize_df, 
                                                                                            plot_kwargs=plot_kwargs)

    (swarmplot_kwargs, barplot_kwargs, sankey_kwargs, violinplot_kwargs, 
     slopegraph_kwargs, reflines_kwargs, legend_kwargs, group_summary_kwargs, redraw_axes_kwargs, 
     delta_dot_kwargs, delta_text_kwargs, summary_bars_kwargs, swarm_bars_kwargs, contrast_bars_kwargs) = get_kwargs(plot_kwargs=plot_kwargs, 
                                                                                                                     ytick_color=ytick_color)

    # We also need to extract the `sankey` and `flow` from the kwargs for plotter.py
    # to use for varying different kinds of paired proportional plots
    # We also don't want to pop the parameter from the kwargs
    sankey = sankey_kwargs["sankey"]
    flow = sankey_kwargs["flow"]
    one_sankey = (
        False if is_paired is not None else None
    )  # Flag to indicate if only one sankey is plotted.
    two_col_sankey = (
        True if proportional and not one_sankey and sankey and not flow else False
    )

    # Extract Color palette
    (color_col, bootstraps_color_by_group, n_groups, 
     swarm_colors, plot_palette_raw, bar_color, 
     plot_palette_bar, plot_palette_contrast, plot_palette_sankey) = get_color_palette(plot_kwargs=plot_kwargs, 
                                                                     plot_data=plot_data, 
                                                                     xvar=xvar, 
                                                                     show_pairs=show_pairs)

    # Initialise the figure.
    fig, rawdata_axes, contrast_axes, swarm_ylim = initialize_fig(plot_kwargs=plot_kwargs, 
                                                                  dabest_obj=dabest_obj, 
                                                                  show_delta2=show_delta2, 
                                                                  show_mini_meta=show_mini_meta, 
                                                                  is_paired=is_paired, 
                                                                  show_pairs=show_pairs, 
                                                                  proportional=proportional, 
                                                                  float_contrast=float_contrast, 
                                                                  face_color=face_color, 
                                                                  )
    
    # Plotting the rawdata.
    if show_pairs:
        temp_idx, temp_all_plot_groups = get_plot_groups(is_paired=is_paired, idx=idx, proportional=proportional, all_plot_groups=all_plot_groups)
        if not proportional:
            # Plot the raw data as a slopegraph.
            slopegraph_plotter(dabest_obj=dabest_obj, 
                               plot_data=plot_data, 
                               xvar=xvar, 
                               yvar=yvar, 
                               color_col=color_col, 
                               plot_palette_raw=plot_palette_raw, 
                               slopegraph_kwargs=slopegraph_kwargs, 
                               rawdata_axes=rawdata_axes, 
                               ytick_color=ytick_color, 
                               temp_idx=temp_idx
                               )

            # DELTA PTS ON CONTRAST PLOT WIP
            show_delta_dots = plot_kwargs["delta_dot"]
            if show_delta_dots and is_paired is not None:
                DeltaDotsPlotter(plot_data=plot_data, 
                                 contrast_axes=contrast_axes, 
                                 delta_id_col=dabest_obj.id_col, 
                                 idx=idx, 
                                 xvar=xvar, 
                                 yvar=yvar, 
                                 is_paired=is_paired, 
                                 color_col=color_col, 
                                 float_contrast=float_contrast, 
                                 plot_palette_raw=plot_palette_raw, 
                                 delta_dot_kwargs=delta_dot_kwargs
                                )

            # Set the tick labels, because the slopegraph plotting doesn't.
            rawdata_axes.set_xticks(np.arange(0, len(temp_all_plot_groups)))
            rawdata_axes.set_xticklabels(temp_all_plot_groups)

        else:
            # Plot the raw data as a set of Sankey Diagrams aligned like barplot.
            sankey_control_group, sankey_test_group = sankeydiag(
                plot_data,
                xvar=xvar,
                yvar=yvar,
                temp_all_plot_groups=temp_all_plot_groups,
                idx=idx,
                temp_idx=temp_idx,
                palette=plot_palette_sankey,
                ax=rawdata_axes,
                **sankey_kwargs
            )
    else:
        if not proportional:
            # Plot the raw data as a swarmplot.
            asymmetric_side = (
                plot_kwargs["swarm_side"] if plot_kwargs["swarm_side"] is not None else "right"
            )  # Default asymmetric side is right

            # swarmplot() plots swarms based on current size of ax
            # Therefore, since the ax size for mini_meta and show_delta changes later on, there has to be increased jitter
            # TODO: to make jitter value more accurate and not just a hardcoded eyeball value
            if show_mini_meta:
                jitter = 1.25
            elif show_delta2:
                jitter = 1.4
            else:
                jitter = 1

            swarmplot_hue = xvar if color_col is None else color_col
            rawdata_plot = swarmplot(
                    data=plot_data,
                    x=xvar,
                    y=yvar,
                    ax=rawdata_axes,
                    order=all_plot_groups,
                    hue=swarmplot_hue,
                    palette=plot_palette_raw,
                    zorder=1,
                    side=asymmetric_side,
                    jitter=jitter,
                    is_drop_gutter=True,
                    gutter_limit=0.45,
                    **swarmplot_kwargs
                )
            if color_col is None:
                rawdata_plot.legend().set_visible(False)

        else:
            # Plot the raw data as a barplot.

            barplotter(xvar=xvar, 
                       yvar=yvar, 
                       all_plot_groups=all_plot_groups, 
                       rawdata_axes=rawdata_axes, 
                       plot_data=plot_data, 
                       bar_color=bar_color, 
                       plot_palette_bar=plot_palette_bar, 
                       plot_kwargs=plot_kwargs, 
                       barplot_kwargs=barplot_kwargs)

        # Plot the error bars.
        if group_summaries is not None:
            if proportional:
                group_summaries_method = "proportional_error_bar"
                group_summaries_offset = 0
                group_summaries_line_color = err_color
            else:
                # Create list to gather xspans.
                xspans = []
                line_colors = []
                for jj, c in enumerate(rawdata_axes.collections):
                    try:
                        if asymmetric_side == "right":
                            # currently offset is hardcoded with value of -0.2
                            x_max_span = -0.2
                        else:
                            _, x_max, _, _ = get_swarm_spans(c)
                            x_max_span = x_max - jj
                        xspans.append(x_max_span)
                    except TypeError:
                        # we have got a None, so skip and move on.
                        pass

                    if bootstraps_color_by_group:
                        line_colors.append(plot_palette_raw[all_plot_groups[jj]])

                    # Break the loop since hue in Seaborn adds collections to axes and it will result in index out of range
                    if jj >= n_groups - 1 and color_col is None:
                        break

                if len(line_colors) != len(all_plot_groups):
                    line_colors = ytick_color

                group_summaries_method = "gapped_lines"
                group_summaries_offset = xspans + np.array(plot_kwargs["group_summaries_offset"])
                group_summaries_line_color = line_colors

            # Plot
            error_bar(
                plot_data,
                x=xvar,
                y=yvar,
                offset=group_summaries_offset,
                line_color=group_summaries_line_color,
                gap_width_percent=1.5,
                type=group_summaries,
                ax=rawdata_axes,
                method=group_summaries_method,
                **group_summary_kwargs
            )

    # Add the counts to the rawdata axes xticks.
    add_counts_to_ticks(plot_data=plot_data, 
                        xvar=xvar, 
                        yvar=yvar, 
                        rawdata_axes=rawdata_axes, 
                        plot_kwargs=plot_kwargs
                        )

    # Save the handles and labels for the legend.
    handles, labels = rawdata_axes.get_legend_handles_labels()
    legend_labels = [l for l in labels]
    legend_handles = [h for h in handles]
    if bootstraps_color_by_group is False:
        rawdata_axes.legend().set_visible(False)

    # Enforce the xtick of rawdata_axes to be 0 and 1 after drawing only one sankey ----> Redundant code
    if one_sankey:
        rawdata_axes.set_xticks([0, 1])

    # Plot effect sizes and bootstraps.
    plot_groups = temp_all_plot_groups if (is_paired == "baseline" and show_pairs and two_col_sankey) else temp_idx if (two_col_sankey) else all_plot_groups

    (ticks_to_skip, ticks_to_plot, ticks_to_skip_contrast, 
     ticks_to_start_twocol_sankey) = extract_contrast_plotting_ticks(is_paired=is_paired, 
                                                                     show_pairs=show_pairs, 
                                                                     two_col_sankey=two_col_sankey, 
                                                                     plot_groups=plot_groups,
                                                                     idx=idx,
                                                                     sankey_control_group=sankey_control_group if two_col_sankey else None,
                                                                    )

    # Plot the bootstraps, then the effect sizes and CIs.
    es_marker_size = plot_kwargs["es_marker_size"]
    halfviolin_alpha = plot_kwargs["halfviolin_alpha"]

    ci_type = plot_kwargs["ci_type"]

    results = effectsize_df.results
    contrast_xtick_labels = []

    (current_group, current_control, 
     current_effsize) = effect_size_curve_plotter(ticks_to_plot=ticks_to_plot, 
                                                  results=results, 
                                                  ci_type=ci_type, 
                                                  contrast_axes=contrast_axes, 
                                                  violinplot_kwargs=violinplot_kwargs, 
                                                  halfviolin_alpha=halfviolin_alpha, 
                                                  ytick_color=ytick_color, 
                                                  es_marker_size=es_marker_size, 
                                                  group_summary_kwargs=group_summary_kwargs, 
                                                  contrast_xtick_labels=contrast_xtick_labels, 
                                                  bootstraps_color_by_group=bootstraps_color_by_group,
                                                  plot_palette_contrast=plot_palette_contrast,
                                                  )

    # Plot mini-meta violin
    if show_mini_meta or show_delta2:
        contrast_xtick_labels = plot_minimeta_or_deltadelta_violins(show_mini_meta=show_mini_meta, 
                                                                    effectsize_df=effectsize_df, 
                                                                    ci_type=ci_type, 
                                                                    rawdata_axes=rawdata_axes,
                                                                    contrast_axes=contrast_axes, 
                                                                    violinplot_kwargs=violinplot_kwargs, 
                                                                    halfviolin_alpha=halfviolin_alpha, 
                                                                    ytick_color=ytick_color, 
                                                                    es_marker_size=es_marker_size, 
                                                                    group_summary_kwargs=group_summary_kwargs, 
                                                                    contrast_xtick_labels=contrast_xtick_labels, 
                                                                    effect_size=effect_size
                                                                    )


    # Make sure the contrast_axes x-lims match the rawdata_axes xlims,
    # and add an extra violinplot tick for delta-delta plot.
    if show_delta2 is False and show_mini_meta is False:
        contrast_axes.set_xticks(rawdata_axes.get_xticks())
    else:
        temp = rawdata_axes.get_xticks()
        temp = np.append(temp, [max(temp) + 1, max(temp) + 2])
        contrast_axes.set_xticks(temp)

    if show_pairs:
        max_x = contrast_axes.get_xlim()[1]
        rawdata_axes.set_xlim(-0.375, max_x)

    if float_contrast:
        contrast_axes.set_xlim(0.5, 1.5)
    elif show_delta2 or show_mini_meta:
        # Increase the xlim of raw data by 2
        temp = rawdata_axes.get_xlim()
        if show_pairs:
            rawdata_axes.set_xlim(temp[0], temp[1] + 0.25)
        else:
            rawdata_axes.set_xlim(temp[0], temp[1] + 2)
        contrast_axes.set_xlim(rawdata_axes.get_xlim())
    else:
        contrast_axes.set_xlim(rawdata_axes.get_xlim())

    # Properly label the contrast ticks.
    for t in ticks_to_skip:
        contrast_xtick_labels.insert(t, "")

    if plot_kwargs["fontsize_contrastxlabel"] is not None:
        fontsize_contrastxlabel = plot_kwargs["fontsize_contrastxlabel"]

    contrast_axes.set_xticklabels(
        contrast_xtick_labels, fontsize=fontsize_contrastxlabel
    )

    if bootstraps_color_by_group is False:
        legend_labels_unique = np.unique(legend_labels)
        unique_idx = np.unique(legend_labels, return_index=True)[1]
        legend_handles_unique = (
            pd.Series(legend_handles, dtype="object").loc[unique_idx]
        ).tolist()

        if len(legend_handles_unique) > 0:
            if float_contrast:
                axes_with_legend = contrast_axes
                if show_pairs:
                    bta = (1.75, 1.02)
                else:
                    bta = (1.5, 1.02)
            else:
                axes_with_legend = rawdata_axes
                if show_pairs:
                    bta = (1.02, 1.0)
                else:
                    bta = (1.0, 1.0)
            leg = axes_with_legend.legend(
                legend_handles_unique,
                legend_labels_unique,
                bbox_to_anchor=bta,
                **legend_kwargs
            )
            if show_pairs:
                for line in leg.get_lines():
                    line.set_linewidth(3.0)

    og_ylim_raw = rawdata_axes.get_ylim()
    og_xlim_raw = rawdata_axes.get_xlim()

    if float_contrast:
        # For Gardner-Altman plots only.

        # Normalize ylims and despine the floating contrast axes.
        # Check that the effect size is within the swarm ylims.
        if effect_size_type in ["mean_diff", "cohens_d", "hedges_g", "cohens_h"]:
            control_group_summary = (
                plot_data.groupby(xvar)
                .mean(numeric_only=True)
                .loc[current_control, yvar]
            )
            test_group_summary = (
                plot_data.groupby(xvar).mean(numeric_only=True).loc[current_group, yvar]
            )
        elif effect_size_type == "median_diff":
            control_group_summary = (
                plot_data.groupby(xvar).median().loc[current_control, yvar]
            )
            test_group_summary = (
                plot_data.groupby(xvar).median().loc[current_group, yvar]
            )

        if swarm_ylim is None:
            swarm_ylim = rawdata_axes.get_ylim()

        _, contrast_xlim_max = contrast_axes.get_xlim()

        difference = float(results.difference[0])

        if effect_size_type in ["mean_diff", "median_diff"]:
            # Align 0 of contrast_axes to reference group mean of rawdata_axes.
            # If the effect size is positive, shift the contrast axis up.
            rawdata_ylims = np.array(rawdata_axes.get_ylim())
            if current_effsize > 0:
                rightmin, rightmax = rawdata_ylims - current_effsize
            # If the effect size is negative, shift the contrast axis down.
            elif current_effsize < 0:
                rightmin, rightmax = rawdata_ylims + current_effsize
            else:
                rightmin, rightmax = rawdata_ylims

            contrast_axes.set_ylim(rightmin, rightmax)

            og_ylim_contrast = rawdata_axes.get_ylim() - np.array(control_group_summary)

            contrast_axes.set_ylim(og_ylim_contrast)
            contrast_axes.set_xlim(contrast_xlim_max - 1, contrast_xlim_max)

        elif effect_size_type in ["cohens_d", "hedges_g", "cohens_h"]:
            if is_paired:
                which_std = 1
            else:
                which_std = 0
            temp_control = plot_data[plot_data[xvar] == current_control][yvar]
            temp_test = plot_data[plot_data[xvar] == current_group][yvar]

            stds = _compute_standardizers(temp_control, temp_test)
            if is_paired:
                pooled_sd = stds[1]
            else:
                pooled_sd = stds[0]

            if effect_size_type == "hedges_g":
                gby_count = plot_data.groupby(xvar).count()
                len_control = gby_count.loc[current_control, yvar]
                len_test = gby_count.loc[current_group, yvar]

                hg_correction_factor = _compute_hedges_correction_factor(
                    len_control, len_test
                )

                ylim_scale_factor = pooled_sd / hg_correction_factor

            elif effect_size_type == "cohens_h":
                ylim_scale_factor = (
                    np.mean(temp_test) - np.mean(temp_control)
                ) / difference

            else:
                ylim_scale_factor = pooled_sd

            scaled_ylim = (
                (rawdata_axes.get_ylim() - control_group_summary) / ylim_scale_factor
            ).tolist()

            contrast_axes.set_ylim(scaled_ylim)
            og_ylim_contrast = scaled_ylim

            contrast_axes.set_xlim(contrast_xlim_max - 1, contrast_xlim_max)

        if one_sankey is None:
            # Draw summary lines for control and test groups..
            for jj, axx in enumerate([rawdata_axes, contrast_axes]):
                # Draw effect size line.
                if jj == 0:
                    ref = control_group_summary
                    diff = test_group_summary
                    effsize_line_start = 1

                elif jj == 1:
                    ref = 0
                    diff = ref + difference
                    effsize_line_start = contrast_xlim_max - 1.1

                xlimlow, xlimhigh = axx.get_xlim()

                # Draw reference line.
                axx.hlines(
                    ref,  # y-coordinates
                    0,
                    xlimhigh,  # x-coordinates, start and end.
                    **reflines_kwargs
                )

                # Draw effect size line.
                axx.hlines(diff, effsize_line_start, xlimhigh, **reflines_kwargs)
        else:
            ref = 0
            diff = ref + difference
            effsize_line_start = contrast_xlim_max - 0.9
            xlimlow, xlimhigh = contrast_axes.get_xlim()
            # Draw reference line.
            contrast_axes.hlines(
                ref,  # y-coordinates
                effsize_line_start,
                xlimhigh,  # x-coordinates, start and end.
                **reflines_kwargs
            )

            # Draw effect size line.
            contrast_axes.hlines(diff, effsize_line_start, xlimhigh, **reflines_kwargs)
        rawdata_axes.set_xlim(og_xlim_raw)  # to align the axis
        # Despine appropriately.
        sns.despine(ax=rawdata_axes, bottom=True)
        sns.despine(ax=contrast_axes, left=True, right=False)

        # Insert break between the rawdata axes and the contrast axes
        # by re-drawing the x-spine.
        rawdata_axes.hlines(
            og_ylim_raw[0],  # yindex
            rawdata_axes.get_xlim()[0],
            1.3,  # xmin, xmax
            **redraw_axes_kwargs
        )
        rawdata_axes.set_ylim(og_ylim_raw)

        contrast_axes.hlines(
            contrast_axes.get_ylim()[0],
            contrast_xlim_max - 0.8,
            contrast_xlim_max,
            **redraw_axes_kwargs
        )

    else:
        # For Cumming Plots only.

        # Set custom contrast_ylim, if it was specified.
        if plot_kwargs["contrast_ylim"] is not None or (
            plot_kwargs["delta2_ylim"] is not None and show_delta2
        ):
            if plot_kwargs["contrast_ylim"] is not None:
                custom_contrast_ylim = plot_kwargs["contrast_ylim"]
                if plot_kwargs["delta2_ylim"] is not None and show_delta2:
                    custom_delta2_ylim = plot_kwargs["delta2_ylim"]
                    if custom_contrast_ylim != custom_delta2_ylim:
                        err1 = "Please check if `contrast_ylim` and `delta2_ylim` are assigned"
                        err2 = "with same values."
                        raise ValueError(err1 + err2)
            else:
                custom_delta2_ylim = plot_kwargs["delta2_ylim"]
                custom_contrast_ylim = custom_delta2_ylim

            if len(custom_contrast_ylim) != 2:
                err1 = "Please check `contrast_ylim` consists of "
                err2 = "exactly two numbers."
                raise ValueError(err1 + err2)

            if effect_size_type == "cliffs_delta":
                # Ensure the ylims for a cliffs_delta plot never exceed [-1, 1].
                l = plot_kwargs["contrast_ylim"][0]
                h = plot_kwargs["contrast_ylim"][1]
                low = -1 if l < -1 else l
                high = 1 if h > 1 else h
                contrast_axes.set_ylim(low, high)
            else:
                contrast_axes.set_ylim(custom_contrast_ylim)

        # If 0 lies within the ylim of the contrast axes,
        # draw a zero reference line.
        contrast_axes_ylim = contrast_axes.get_ylim()
        if contrast_axes_ylim[0] < contrast_axes_ylim[1]:
            contrast_ylim_low, contrast_ylim_high = contrast_axes_ylim
        else:
            contrast_ylim_high, contrast_ylim_low = contrast_axes_ylim
        if contrast_ylim_low < 0 < contrast_ylim_high:
            contrast_axes.axhline(y=0, **reflines_kwargs)

        if is_paired == "baseline" and show_pairs:
            if two_col_sankey:
                rightend_ticks_raw = np.array([len(i) - 2 for i in idx]) + np.array(
                    ticks_to_start_twocol_sankey
                )
            elif proportional and is_paired is not None:
                rightend_ticks_raw = np.array([len(i) - 1 for i in idx]) + np.array(
                    ticks_to_skip
                )
            else:
                rightend_ticks_raw = np.array(
                    [len(i) - 1 for i in temp_idx]
                ) + np.array(ticks_to_skip)
            for ax in [rawdata_axes]:
                sns.despine(ax=ax, bottom=True)

                ylim = ax.get_ylim()
                xlim = ax.get_xlim()
                redraw_axes_kwargs["y"] = ylim[0]

                if two_col_sankey:
                    for k, start_tick in enumerate(ticks_to_start_twocol_sankey):
                        end_tick = rightend_ticks_raw[k]
                        ax.hlines(xmin=start_tick, xmax=end_tick, **redraw_axes_kwargs)
                else:
                    for k, start_tick in enumerate(ticks_to_skip):
                        end_tick = rightend_ticks_raw[k]
                        ax.hlines(xmin=start_tick, xmax=end_tick, **redraw_axes_kwargs)
                ax.set_ylim(ylim)
                del redraw_axes_kwargs["y"]

            if not proportional:
                temp_length = [(len(i) - 1) for i in idx]
            else:
                temp_length = [(len(i) - 1) * 2 - 1 for i in idx]
            if two_col_sankey:
                rightend_ticks_contrast = np.array(
                    [len(i) - 2 for i in idx]
                ) + np.array(ticks_to_start_twocol_sankey)
            elif proportional and is_paired is not None:
                rightend_ticks_contrast = np.array(
                    [len(i) - 1 for i in idx]
                ) + np.array(ticks_to_skip)
            else:
                rightend_ticks_contrast = np.array(temp_length) + np.array(
                    ticks_to_skip_contrast
                )
            for ax in [contrast_axes]:
                sns.despine(ax=ax, bottom=True)

                ylim = ax.get_ylim()
                xlim = ax.get_xlim()
                redraw_axes_kwargs["y"] = ylim[0]

                if two_col_sankey:
                    for k, start_tick in enumerate(ticks_to_start_twocol_sankey):
                        end_tick = rightend_ticks_contrast[k]
                        ax.hlines(xmin=start_tick, xmax=end_tick, **redraw_axes_kwargs)
                else:
                    for k, start_tick in enumerate(ticks_to_skip_contrast):
                        end_tick = rightend_ticks_contrast[k]
                        ax.hlines(xmin=start_tick, xmax=end_tick, **redraw_axes_kwargs)

                ax.set_ylim(ylim)
                del redraw_axes_kwargs["y"]
        else:
            # Compute the end of each x-axes line.
            if two_col_sankey:
                rightend_ticks = np.array([len(i) - 2 for i in idx]) + np.array(
                    ticks_to_start_twocol_sankey
                )
            else:
                rightend_ticks = np.array([len(i) - 1 for i in idx]) + np.array(
                    ticks_to_skip
                )

            for ax in [rawdata_axes, contrast_axes]:
                sns.despine(ax=ax, bottom=True)

                ylim = ax.get_ylim()
                xlim = ax.get_xlim()
                redraw_axes_kwargs["y"] = ylim[0]

                if two_col_sankey:
                    for k, start_tick in enumerate(ticks_to_start_twocol_sankey):
                        end_tick = rightend_ticks[k]
                        ax.hlines(xmin=start_tick, xmax=end_tick, **redraw_axes_kwargs)
                else:
                    for k, start_tick in enumerate(ticks_to_skip):
                        end_tick = rightend_ticks[k]
                        ax.hlines(xmin=start_tick, xmax=end_tick, **redraw_axes_kwargs)

                ax.set_ylim(ylim)
                del redraw_axes_kwargs["y"]

    if show_delta2 or show_mini_meta:
        ylim = contrast_axes.get_ylim()
        redraw_axes_kwargs["y"] = ylim[0]
        x_ticks = contrast_axes.get_xticks()
        contrast_axes.hlines(xmin=x_ticks[-2], xmax=x_ticks[-1], **redraw_axes_kwargs)
        del redraw_axes_kwargs["y"]

    # Set raw axes y-label.
    swarm_label = plot_kwargs["swarm_label"]
    if swarm_label is None and yvar is None:
        swarm_label = "value"
    elif swarm_label is None and yvar is not None:
        swarm_label = yvar

    bar_label = plot_kwargs["bar_label"]
    if bar_label is None and effect_size_type != "cohens_h":
        bar_label = "proportion of success"
    elif bar_label is None and effect_size_type == "cohens_h":
        bar_label = "value"

    # Place contrast axes y-label.
    contrast_label_dict = {
        "mean_diff": "mean difference",
        "median_diff": "median difference",
        "cohens_d": "Cohen's d",
        "hedges_g": "Hedges' g",
        "cliffs_delta": "Cliff's delta",
        "cohens_h": "Cohen's h",
        "delta_g": "mean difference",
    }

    if proportional and effect_size_type != "cohens_h":
        default_contrast_label = "proportion difference"
    elif effect_size_type == "delta_g":
        default_contrast_label = "Hedges' g"
    else:
        default_contrast_label = contrast_label_dict[effectsize_df.effect_size]

    if plot_kwargs["contrast_label"] is None:
        if is_paired:
            contrast_label = "paired\n{}".format(default_contrast_label)
        else:
            contrast_label = default_contrast_label
        contrast_label = contrast_label.capitalize()
    else:
        contrast_label = plot_kwargs["contrast_label"]

    if plot_kwargs["fontsize_rawylabel"] is not None:
        fontsize_rawylabel = plot_kwargs["fontsize_rawylabel"]
    if plot_kwargs["fontsize_contrastylabel"] is not None:
        fontsize_contrastylabel = plot_kwargs["fontsize_contrastylabel"]
    if plot_kwargs["fontsize_delta2label"] is not None:
        fontsize_delta2label = plot_kwargs["fontsize_delta2label"]

    contrast_axes.set_ylabel(contrast_label, fontsize=fontsize_contrastylabel)
    if float_contrast:
        contrast_axes.yaxis.set_label_position("right")

    # Set the rawdata axes labels appropriately
    if not proportional:
        rawdata_axes.set_ylabel(swarm_label, fontsize=fontsize_rawylabel)
    else:
        rawdata_axes.set_ylabel(bar_label, fontsize=fontsize_rawylabel)
    rawdata_axes.set_xlabel("")

    # Because we turned the axes frame off, we also need to draw back
    # the y-spine for both axes.
    if not float_contrast:
        rawdata_axes.set_xlim(contrast_axes.get_xlim())
    og_xlim_raw = rawdata_axes.get_xlim()
    rawdata_axes.vlines(
        og_xlim_raw[0], og_ylim_raw[0], og_ylim_raw[1], **redraw_axes_kwargs
    )

    og_xlim_contrast = contrast_axes.get_xlim()

    if float_contrast:
        xpos = og_xlim_contrast[1]
    else:
        xpos = og_xlim_contrast[0]

    og_ylim_contrast = contrast_axes.get_ylim()
    contrast_axes.vlines(
        xpos, og_ylim_contrast[0], og_ylim_contrast[1], **redraw_axes_kwargs
    )

    if show_delta2:
        if plot_kwargs["delta2_label"] is not None:
            delta2_label = plot_kwargs["delta2_label"]
        elif effect_size == "mean_diff":
            delta2_label = "delta - delta"
        else:
            delta2_label = "deltas' g"
        delta2_axes = contrast_axes.twinx()
        delta2_axes.set_frame_on(False)
        delta2_axes.set_ylabel(delta2_label, fontsize=fontsize_delta2label)
        og_xlim_delta = contrast_axes.get_xlim()
        og_ylim_delta = contrast_axes.get_ylim()
        delta2_axes.set_ylim(og_ylim_delta)
        delta2_axes.vlines(
            og_xlim_delta[1], og_ylim_delta[0], og_ylim_delta[1], **redraw_axes_kwargs
        )

    ################################################### GRIDKEY  WIP
    # if gridkey_rows is None, skip everything here
    gridkey_rows = plot_kwargs["gridkey_rows"]
    if gridkey_rows is not None:
        grid_key_WIP(is_paired=is_paired, 
                     idx=idx, 
                     all_plot_groups=all_plot_groups, 
                     gridkey_rows=gridkey_rows, 
                     rawdata_axes=rawdata_axes, 
                     contrast_axes=contrast_axes,
                     plot_data=plot_data, 
                     xvar=xvar, 
                     yvar=yvar, 
                     results=results, 
                     show_delta2=show_delta2, 
                     show_mini_meta=show_mini_meta, 
                     float_contrast=float_contrast,
                     plot_kwargs=plot_kwargs,
                     )

    ################################################### Swarm & Contrast & Summary Bars & Delta text WIP
    # Swarm bars WIP
    swarm_bars = plot_kwargs["swarm_bars"]
    if swarm_bars and not proportional:
        swarm_bars_plotter(plot_data=plot_data, 
                           xvar=xvar, 
                           yvar=yvar, 
                           ax=rawdata_axes, 
                           swarm_bars_kwargs=swarm_bars_kwargs, 
                           color_col=color_col, 
                           swarm_colors=swarm_colors, 
                           is_paired=is_paired
                           )

    # Contrast bars WIP
    contrast_bars = plot_kwargs["contrast_bars"]
    if contrast_bars:
        contrast_bars_plotter(results=results, 
                              ax_to_plot=contrast_axes, 
                              swarm_plot_ax=rawdata_axes,
                              ticks_to_plot=ticks_to_plot, 
                              contrast_bars_kwargs=contrast_bars_kwargs, 
                              color_col=color_col, 
                              swarm_colors=swarm_colors, 
                              show_mini_meta=show_mini_meta, 
                              mini_meta_delta=effectsize_df.mini_meta_delta if show_mini_meta else None, 
                              show_delta2=show_delta2, 
                              delta_delta=effectsize_df.delta_delta if show_delta2 else None, 
                              proportional=proportional, 
                              is_paired=is_paired
                              )

    # Summary bars WIP
    summary_bars = plot_kwargs["summary_bars"]
    if summary_bars is not None:
        summary_bars_plotter(summary_bars=summary_bars, 
                             results=results, 
                             ax_to_plot=contrast_axes, 
                             float_contrast=float_contrast,
                             summary_bars_kwargs=summary_bars_kwargs, 
                             ci_type=ci_type, 
                             ticks_to_plot=ticks_to_plot, 
                             color_col=color_col,
                             swarm_colors=swarm_colors, 
                             proportional=proportional, 
                             is_paired=is_paired
                             )
    # Delta text WIP
    delta_text = plot_kwargs["delta_text"]
    if delta_text: 
        delta_text_plotter(results=results, 
                           ax_to_plot=contrast_axes, 
                           swarm_plot_ax=rawdata_axes, 
                           ticks_to_plot=ticks_to_plot, 
                           delta_text_kwargs=delta_text_kwargs, 
                           color_col=color_col, 
                           swarm_colors=swarm_colors, 
                           is_paired=is_paired,
                           proportional=proportional, 
                           float_contrast=float_contrast, 
                           show_mini_meta=show_mini_meta, 
                           mini_meta_delta=effectsize_df.mini_meta_delta if show_mini_meta else None, 
                           show_delta2=show_delta2, 
                           delta_delta=effectsize_df.delta_delta if show_delta2 else None
                           )
    ################################################### Swarm & Contrast & Summary Bars & Delta text WIP END

    # Make sure no stray ticks appear!
    rawdata_axes.xaxis.set_ticks_position("bottom")
    rawdata_axes.yaxis.set_ticks_position("left")
    contrast_axes.xaxis.set_ticks_position("bottom")
    if float_contrast is False:
        contrast_axes.yaxis.set_ticks_position("left")

    # Reset rcParams.
    for parameter in _changed_rcParams:
        plt.rcParams[parameter] = original_rcParams[parameter]

    # Return the figure.
    return fig
